## インポート

In [1]:

import sys
import re
import difflib
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn import linear_model
from sklearn.metrics import (mean_absolute_error,
                             mean_squared_error,
                             r2_score)
from sklearn.model_selection import train_test_split
from sklearn.externals import joblib

from config import (NUMBERS,
                                   NUMBERS3,
                                   NUMBERS4,
                                   NUMBERS4_RESULT,
                                   NUMBERS4_VECTORS,
                                   NUMBERS4_PATTERNS)
import utils


%load_ext autoreload
%autoreload 2

#### 設定

In [550]:
OLD_WEEK = "仏滅"
TYPE            = "-"

In [703]:
PATTERN = "pattern{0}".format(9)

##### 特徴量

In [704]:
VEC = utils.get_vectors(OLD_WEEK, TYPE, NUMBERS4_VECTORS)
NUM  = utils.get_patterns(NUMBERS4_PATTERNS, PATTERN, TYPE)
print(NUM, VEC)

[1, 2] [0, 1, 3]


##### セット

In [705]:
X       = []
Y       = []
TIME = []
k1 = VEC[0]
k2 = VEC[1]
k3 = VEC[2]
n1 = NUM[0]
n2 = NUM[1]

##### 計算

In [706]:
X = []
Y = []
TIME = []
for _i in range(len(NUMBERS)):
    _time = NUMBERS[_i][0]
    _y = NUMBERS[_i][3]
    _x1 = NUMBERS4[_i][_time][k1]
    _x2 = NUMBERS4[_i][_time][k2]
    _x３ = NUMBERS4[_i][_time][k3]
    _z = NUMBERS[_i][4]
    if not len(list(set(_y) & set(_x1))) >= n1 and not len(list(set(_y) & set(_x2))) >= n2  and _z == OLD_WEEK:
        Y.append(_y)
        X.append([_x1, _x2, _x3])
        TIME.append(_time)

## 予測

In [707]:
y = pd.DataFrame(Y)
x = pd.DataFrame(X)

In [708]:
len(y)

23

In [709]:
len(TIME)

23

In [710]:
model = linear_model.LinearRegression()
model.fit(X, Y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [711]:
px = x
py = model.predict(px)

In [712]:
for _i in range(len(py)):
    _py = str(int(py[_i]))
    for _k, _v in enumerate(NUMBERS):
        if _v[0] == TIME[_i]:
            if len(list(set(_v[3]) & set(_py))) >= 3:
                print(TIME[_i], _v[3], py[_i])
            # print(TIME[_i], _v[3], py[_i])

4617 3806 3980.424340566895


## 決定係数

In [713]:
r2_score(y, py)

0.0818439630581661

In [714]:
model.score(x, y)

0.0818439630581661

In [715]:
model.coef_[0]

0.1787499031819446

## 予測

In [716]:
pkl_file = "data/numbers4_20180717.pkl"

In [717]:
data = [float("6817"), float("1963"), float("6830")]

In [718]:
joblib.dump(model, pkl_file)

['data/numbers4_20180717.pkl']

In [719]:
clf = joblib.load(pkl_file)

In [720]:
processor = clf.predict([data])

In [721]:
processor

array([6178.15644065])